## Clone the Repository

This notebook is modified from a Colab tutorial notebook that was written for this repo.

In [1]:
!git clone https://github.com/sovit-123/fastercnn-pytorch-training-pipeline.git

Cloning into 'fastercnn-pytorch-training-pipeline'...
remote: Enumerating objects: 1421, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 1421 (delta 354), reused 312 (delta 311), pack-reused 984 (from 2)
Receiving objects: 100% (1421/1421), 12.85 MiB | 33.74 MiB/s, done.
Resolving deltas: 100% (957/957), done.


In [2]:
# Enter the repo directory.
%cd fastercnn-pytorch-training-pipeline/

/content/fastercnn-pytorch-training-pipeline


In [3]:
# Install the Requirements
!pip install -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.7/458.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━

## Create the Custom Dataset YAML File.

1. Download the dataset from RoboFlow.
2. From this data create the YAML file.

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HhNnM9lzMEeBCZh4mD5x")
project = rf.workspace("plastic-detection-r16is").project("underwater_plastics_og_data")
version = project.version(1)
dataset = version.download("voc")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to underwater_plastics_og_data-1 in voc:: 100%|██████████| 13575/13575 [00:01<00:00, 8013.09it/s]


In [2]:
# Write the YAML file
%%writefile /content/fastercnn-pytorch-training-pipeline/data_configs/custom_data.yaml
TRAIN_DIR_IMAGES: '/content/underwater_plastics_og_data-1/train'
TRAIN_DIR_LABELS: '/content/underwater_plastics_og_data-1/train'
VALID_DIR_IMAGES: '/content/underwater_plastics_og_data-1/valid'
VALID_DIR_LABELS: '/content/underwater_plastics_og_data-1/valid'

# Class names.
CLASSES: [
    'rov',
    'plant', 'animal_fish', 'animal_starfish',
    'animal_shells', 'animal_crab', 'animal_eel',
    'animal_etc', 'trash_clothing', 'trash_pipe', 'trash_bottle',
    'trash_bag', 'trash_snack_wrapper', 'trash_can', 'trash_cup',
    'trash_container', 'trash_unknown_instance', 'trash_branch',
    'trash_wreckage', 'trash_tarp', 'trash_rope', 'trash_net'
]

# Number of classes.
NC: 22

# Whether to save the predictions of the validation set while training.
SAVE_VALID_PREDICTION_IMAGES: True

Writing /content/fastercnn-pytorch-training-pipeline/data_configs/custom_data.yaml


## Running Inference

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Upload best_model.pth for weights from training.
!python /content/fastercnn-pytorch-training-pipeline/eval.py --weights "/content/drive/MyDrive/Computer Vision Final Project/Faster_RCNN/best_model.pth" --data /content/fastercnn-pytorch-training-pipeline/data_configs/custom_data.yaml --model fasterrcnn_resnet50_fpn_v2 --verbose

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100% 167M/167M [00:00<00:00, 231MB/s]
Checking Labels and images...
100% 1087/1087 [00:00<00:00, 4567.45it/s]
Removed 0 problematic images and annotations.
  1% 1/136 [00:50<1:54:21, 50.83s/it]INFO:utils.logging:Test:  [  0/136]  eta: 1:55:12    time: 50.8296  data: 0.5438
 15% 20/136 [16:34<1:36:41, 50.02s/it]